Importing all the needed libraries

In [ ]:
import pandas as pd
import numpy as np
import re
!pip install rapidfuzz
from rapidfuzz import fuzz, process
!pip install unidecode
from unidecode import unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 4.5 MB/s eta 0:00:00


Importing dataset that I will be working with, Skipping the 4 rows of metadata,

In [ ]:
# 📄 Step 3: Load "All Faculty Report for Wellness Survey" CSV and skip the first 5 rows (header starts on row 6)
file_path = '/content/All Faculty Report for Wellness Survey_2024-06-28(ALL Faculy forWellness Survey).csv'

try:
    df = pd.read_csv(file_path, skiprows=5, encoding='utf-8')  # You can try 'latin1' if utf-8 fails
    print(f"✅ Loaded successfully! Shape: {df.shape}")
except UnicodeDecodeError:
    df = pd.read_csv(file_path, skiprows=5, encoding='latin1')
    print("⚠️ Encoding fallback used: latin1")
    print(f"✅ Loaded successfully! Shape: {df.shape}")

# Preview first few rows
df.head()


✅ Loaded successfully! Shape: (2594, 39)


,ID,LASTNAME,FIRSTNAME,MIDDLENAME,EMAILADDRESS,NETID,NPI_NUMBER,PRIMARYSCHOOL,Department/Division,INSTITUTE,...,Joint-2 Department,Joint-2 Percent_Effort,Joint-2 Campus,Joint School-3,Joint-3 Start Date,Joint-3 Rank,Joint-3 Title,Joint-3 Department,Joint-3 Percent_Effort,Joint-3 Campus
0,00152094,Abadiotakis,Elizabeth,NaN,ea626@rutgers.edu,EA626,NaN,School of Health Professions,Psychiatric Rehabilitation & Counseling Profes...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00044160,Abazia,Daniel,T,dabazia@pharmacy.rutgers.edu,DABAZIA,NaN,Ernest Mario School of Pharmacy,Pharmacy Practice & Administration,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00101428,Abbas,Ali,M.,abbasal@rutgers.edu,ABBASAL,1194948604,Rutgers School of Dental Medicine,Diagnostic Sciences,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00106433,Abdellatif,Maha,NaN,abdellma@rutgers.edu,ABDELLMA,NaN,New Jersey Medical School,Cell Biology & Molecular Medicine,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00108059,Abdelquader,Hammad,NaN,habdel@rutgers.edu,HABDEL,1386096865,Robert Wood Johnson Medical School,Family Medicine and Community Health,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 📄 Load Rutgers HG Employee List
hg_file_path = '/content/RutgersHGEmployeeList.xlsx - Export Worksheet.csv'

try:
    hg_df = pd.read_csv(hg_file_path, encoding='utf-8')  # utf-8 default
    print(f"✅ HG Employee List loaded successfully! Shape: {hg_df.shape}")
except UnicodeDecodeError:
    hg_df = pd.read_csv(hg_file_path, encoding='latin1')  # fallback encoding
    print("⚠️ Encoding fallback used: latin1")
    print(f"✅ HG Employee List loaded successfully! Shape: {hg_df.shape}")

# Preview the first few rows
hg_df.head()


✅ HG Employee List loaded successfully! Shape: (8007, 26)


,EMPLOYEE_ID,NETID,EMPLOYEE_NAME,LAST_NAME,FIRST_NAME,MIDDLE_INIT,JOB_TITLE,PRIME_IND,CURRENT_FLAG,FAC_STAFF_IND,...,DEPT_LEVEL3,DEPT_LEVEL4,DEPT_LEVEL5,EMPLOYMENT_STATUS_CODE,EMPLOYEE_CLASS,TENURE_DATE,PREFERRED_EMAIL_ADDR,CAMPUS_EMAIL_ADDR,NPI_NUMBER,FTE
0,187584,YA374,"AMBO,YUKO",AMBO,YUKO,NaN,VISITING RESEARCHER,Y,Y,Staff,...,RWJ-Finance Office,RWJ-Child Health Institute,NaN,A,3,NaN,yu.jan.115@gmail.com,yuko.ambo@rutgers.edu,NaN,1.0
1,179003,ABADGB,"Abad,Grace",Abad,Grace,NaN,ADVANCED PRACTICE NURSE,Y,Y,Staff,...,UBHC-Administration,UBHC-Acute Psych Services,NaN,A,1,NaN,chriesella@yahoo.com,abadgb@rutgers.edu,NaN,1.0
2,103926,ABADVA,"Abad,Valori",Abad,Valori,NaN,REGIONAL NURSE ADMINISTRATOR,Y,Y,Staff,...,SN-Admin (Dean's Office),SN-FXB Center-Chp Program,NaN,A,1,NaN,NaN,valori.abad@rutgers.edu,NaN,1.0
3,103926,ABADVA,"Abad,Valori",Abad,Valori,NaN,REGIONAL NURSE ADMINISTRATOR,Y,Y,Staff,...,SN-Admin (Dean's Office),SN-FXB Center-Chp Program,NaN,A,1,NaN,NaN,valori.abad@rutgers.edu,NaN,1.0
4,103926,ABADVA,"Abad,Valori",Abad,Valori,NaN,REGIONAL NURSE ADMINISTRATOR,Y,Y,Staff,...,SN-Admin (Dean's Office),SN-FXB Center-Chp Program,NaN,A,1,NaN,NaN,valori.abad@rutgers.edu,NaN,1.0


Step 4: Clean & Standardize Column Names
This will:

Strip spaces

Lowercase all column headers

Replace spaces with underscores

Remove special characters

This makes your DataFrame easier to work with in all future steps.


In [ ]:
# 🔧 Function to clean column names
def clean_column_names(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(' ', '_')
        .str.replace(r'[^\w\s]', '', regex=True)
    )
    return df

# Apply to both dataframes
df = clean_column_names(df)
hg_df = clean_column_names(hg_df)

# Show cleaned column names
print("✅ Cleaned column names for df:\n", df.columns.tolist())
print("\n✅ Cleaned column names for hg_df:\n", hg_df.columns.tolist())


✅ Cleaned column names for df:
 ['id', 'lastname', 'firstname', 'middlename', 'emailaddress', 'netid', 'npi_number', 'primaryschool', 'departmentdivision', 'institute', 'primaryfacultytrack', 'academictitle', 'put', 'employeeclasscode', 'fte', 'salarytable', 'salarygrade', 'clinicalnonclinical', 'joint_school1', 'joint1_start_date', 'joint1_rank', 'joint1_title', 'joint1_department', 'joint1_percent_effort', 'joint1_campus', 'joint_school2', 'joint2_start_date', 'joint2_rank', 'joint2_title', 'joint2_department', 'joint2_percent_effort', 'joint2_campus', 'joint_school3', 'joint3_start_date', 'joint3_rank', 'joint3_title', 'joint3_department', 'joint3_percent_effort', 'joint3_campus']

✅ Cleaned column names for hg_df:
 ['employee_id', 'netid', 'employee_name', 'last_name', 'first_name', 'middle_init', 'job_title', 'prime_ind', 'current_flag', 'fac_staff_ind', 'campus', 'udo', 'dept_code', 'dept_descr', 'dept_level1', 'dept_level2', 'dept_level3', 'dept_level4', 'dept_level5', 'employme

Step 5: Remove Duplicates & Normalize Identifiers (netid, email, name)
We’ll clean and deduplicate hg_df, your main employee list, by:

🔁 Dropping exact row duplicates

👥 Deduplicating by netid (keep the most complete record)

✂️ Stripping spaces + fixing casing in key identifier columns (netid, email, first_name, last_name)

In [ ]:
# 📦 Step 5: Clean & Deduplicate hg_df

# 1. Drop exact duplicate rows (same across all columns)
hg_df = hg_df.drop_duplicates()
print(f"✅ Dropped exact duplicates — new shape: {hg_df.shape}")

# 2. Clean NETID and EMAIL columns
for col in ['netid', 'campus_email_addr', 'preferred_email_addr']:
    if col in hg_df.columns:
        hg_df[col] = hg_df[col].astype(str).str.strip().str.lower()

# 3. Clean FIRST and LAST names
for col in ['first_name', 'last_name']:
    if col in hg_df.columns:
        hg_df[col] = hg_df[col].astype(str).str.strip().str.title()

# 4. Deduplicate based on NETID — keep the first or most complete
if 'netid' in hg_df.columns:
    hg_df = hg_df.sort_values(by='netid')  # Optional: sort to keep specific records
    hg_df = hg_df.drop_duplicates(subset='netid', keep='first')
    print(f"✅ Deduplicated on netid — new shape: {hg_df.shape}")

# ✅ Preview cleaned data
hg_df.head()


✅ Dropped exact duplicates — new shape: (8007, 26)
✅ Deduplicated on netid — new shape: (5028, 26)


,employee_id,netid,employee_name,last_name,first_name,middle_init,job_title,prime_ind,current_flag,fac_staff_ind,...,dept_level3,dept_level4,dept_level5,employment_status_code,employee_class,tenure_date,preferred_email_addr,campus_email_addr,npi_number,fte
330,154389,aa1014,"Athanasakis,Andreas",Athanasakis,Andreas,NaN,CMC OPERATOR I,Y,Y,Staff,...,CINJ-Director's Office,CINJ-GMP Facility,NaN,A,1,NaN,athanasakisandreas@yaho..com,aa1014@research.rutgers.edu,NaN,1.0
58,182194,aa1330,"Adedeji,Anuoluwapo J",Adedeji,Anuoluwapo,J,POST DOCTORAL ASSOC,Y,Y,Faculty,...,Ernest Mario School - Pharmacy,Pharm-Pharmacy Practice&Admin,NaN,A,1,NaN,anujoyadedeji@gmail.com,joy.adedeji@rutgers.edu,NaN,1.0
264,78203,aa1576,"Antony Jude,Anthonypillai Joseph",Antony Jude,Anthonypillai Joseph,NaN,ASST RESEARCH PROFESSOR CAL YR,Y,Y,Faculty,...,Ernest Mario School - Pharmacy,Pharm-Pharmacology&Toxicology,NaN,A,1,NaN,aantony@mail.med.upenn.edu,joseph.antonyjude@rutgers.edu,NaN,1.0
146,126370,aa1917,"Alghali-Ano,Adisa",Alghali-Ano,Adisa,NaN,PROG MGR,Y,Y,Staff,...,UBHC-Administration,UBHC-Acute Psych Services,UBHC-Inpatient/CTR,A,1,NaN,adisaalghali@gmail.com,aa1917@rutgers.edu,NaN,1.0
351,159930,aa2447,"Avula,Amulya",Avula,Amulya,NaN,HLTH PROGRAM ANALYST,Y,Y,Staff,...,NJMS-Dean's Office,NJMS-Medicine Admin,NJMS-Medicine-Infectious Disea,A,1,NaN,amulyaavula5@gmail.com,amulya.avula@rutgers.edu,NaN,1.0


Why This Is Important:
Removes clutter and duplicated staff

Ensures every netid (your primary key) appears only once

Cleans up fields you’ll use for matching/classification in the next steps



In [ ]:
# If you uploaded via the upload widget, use the actual filenames you see in your file list

slg_clinical_path = "/content/Final SLG_Clinical.csv"
slg_nonclinical_path = "/content/Final SLG_Non-Clinical.csv"
physician_survey_path = "/content/Physician-Survey(Sheet1).csv"

# Try reading each file with utf-8 and fallback to latin1 if needed
try:
    slg_clinical_df = pd.read_csv(slg_clinical_path, encoding='utf-8')
except:
    slg_clinical_df = pd.read_csv(slg_clinical_path, encoding='latin1')

try:
    slg_nonclinical_df = pd.read_csv(slg_nonclinical_path, encoding='utf-8')
except:
    slg_nonclinical_df = pd.read_csv(slg_nonclinical_path, encoding='latin1')

try:
    physician_df = pd.read_csv(physician_survey_path, encoding='utf-8')
except:
    physician_df = pd.read_csv(physician_survey_path, encoding='latin1')

# Preview
print("✅ SLG Clinical:")
display(slg_clinical_df.head())

print("✅ SLG Non-Clinical:")
display(slg_nonclinical_df.head())

print("✅ Physician Survey:")
display(physician_df.head())


✅ SLG Clinical:


,Email Address,Lname,Fname,UBHC_UCHC,School,Department/School Division,Department/School Division_Revised_A,Department/School Division_Revised_B,Institute,Track/Tenure,...,Secondary Email,Middle Name,UBHC_UCHC Management_Non-Management,UBHC_UCHC_DISCIPLINE,ID,NPI_NUMBER,NetID,VERGE_UNIQUE_ID,SIY,Unnamed: 26
0,as4266@rutgers.edu,Schwartz,Anna,UBHC,Staff New Jersey Medical School,NaN,,,NaN,NaN,...,NaN,Miriam,NON-MANAGEMENT,SOCIAL WORKER,UBHC_UCHC_747,NaN,AS4266,NaN,SIY,NaN
1,shoshana.sperling@rutgers.edu,Sperling,Shoshana,UBHC,Staff New Jersey Medical School,NaN,,,NaN,NaN,...,NaN,NaN,NON-MANAGEMENT,PSYCHOLOGIST,UBHC_UCHC_942,NaN,SS3943,NaN,SIY,NaN
2,iyabo.aasa@rutgers.edu,Aasa,Iyabo,UCHC,NaN,NaN,,,NaN,NaN,...,NaN,Adeola,NON-MANAGEMENT,NURSE,UBHC_UCHC_1,NaN,AASALA,NaN,NaN,NaN
3,abbottgl@rutgers.edu,Abbott,Gerald,UCHC,NaN,NaN,,,NaN,NaN,...,NaN,L.,NON-MANAGEMENT,DENTIST,UBHC_UCHC_4,NaN,ABBOTTGL,NaN,NaN,NaN
4,abdullah.abdulhakeem@rutgers.edu,Abdul-Hakeem,Abdullah,UBHC,NaN,NaN,,,NaN,NaN,...,NaN,NaN,NON-MANAGEMENT,NaN,UBHC_UCHC_6,NaN,ABDULHAB,NaN,NaN,NaN


✅ SLG Non-Clinical:


,Email Address,Lname,Fname,UBHC_UCHC,School,Department/School Division,Department/School Division_Revised_A,Department/School Division_Revised_B,Institute,Track/Tenure,...,FTE,Secondary Email,Middle Name,UBHC_UCHC Management_Non-Management,UBHC_UCHC_DISCIPLINE,ID,NPI_NUMBER,NetID,VERGE_UNIQUE_ID,SIY
0,agv28@rutgers.edu,Vatavuk,Anita,UBHC,NaN,NaN,,NaN,NaN,NaN,...,1.0,NaN,NaN,NON-MANAGEMENT,NaN,UBHC_UCHC_1487,NaN,AGV28,NaN,NaN
1,reine.abita@rutgers.edu,Abita,Reine,UCHC,NaN,NaN,,NaN,NaN,NaN,...,0.5,NaN,NaN,NON-MANAGEMENT,NaN,UBHC_UCHC_1488,NaN,RAA202,NaN,NaN
2,alexandra.figueroa@rutgers.edu,Figueroa,Alexandra,UBHC,NaN,NaN,,NaN,NaN,NaN,...,1.0,NaN,NaN,NON-MANAGEMENT,NaN,UBHC_UCHC_1489,NaN,FIGUERA1,NaN,NaN
3,mitch.abrams@rutgers.edu,Abrams,Mitchell,UCHC,NaN,NaN,,NaN,NaN,NaN,...,1.0,NaN,NaN,MANAGEMENT,NaN,UBHC_UCHC_1490,NaN,ABRAMSMI,NaN,NaN
4,boswelmj@ubhc.rutgers.edu,Boswell,Megan,UBHC,NaN,NaN,,NaN,NaN,NaN,...,1.0,NaN,NaN,NON-MANAGEMENT,NaN,UBHC_UCHC_1491,NaN,BOSWELMJ,NaN,NaN


✅ Physician Survey:


,VERGE_UNIQUE_ID,LAST_NAME,FIRST_NAME,MIDDLE_NAME,PRIMARY_EMAIL,NPI_NUMBER,DEPARTMENT,PRACTITIONER_TYPE,STAFF_STATUS,HOSPITAL_AFFILIATION_NAME,DATA_ENTRY_DATE_FOR_HOSPITAL_AFFLIATION_IN_COLUMN_K (Initial Appointment Date)
0,1974797,Aamer,Aaiza,NaN,aaiza.aamer@gmail.com,1.013370e+09,Medicine,Physician,Active,Jersey City Medical Center,10/22/2020
1,1756499,Aaron,Michael,NaN,heartdo@aol.com,1.336129e+09,Cardiology,Physician,Attending,Community Medical Center,9/8/1994
2,1823933,Abad,Grace,NaN,chriesella@yahoo.com,1.710203e+09,Family Medicine,Advanced Nurse Practitioner,Ancillary,RWJ - Somerset,10/26/2010
3,2464407,Abadeer,Ann Marie,NaN,annmarie.abadeer@gmail.com,1.437796e+09,Surgery,Physician Assistant,Ancillary Staff,Newark Beth Israel Medical Center,1/11/2023
4,2129794,Abadinas,Ria Charisma,Samson,rcabadinas@gmail.com,1.396356e+09,Emergency Medicine,Advanced Nurse Practitioner,Allied Health Professional,Jersey City Medical Center,6/17/2021


In [ ]:
# 🏷️ Step 6: Add Clinical/Non-Clinical Classification Column

# Clean comparison fields
def normalize_column(df, col):
    return df[col].astype(str).str.strip().str.lower()

# Clean netid/email in all DataFrames
for df_to_clean in [hg_df, slg_clinical_df, slg_nonclinical_df, physician_df]:
    for col in ['netid', 'campus_email_addr', 'preferred_email_addr']:
        if col in df_to_clean.columns:
            df_to_clean[col] = normalize_column(df_to_clean, col)

# Create a new column to store classification result
hg_df['clinical_classification'] = 'Unclassified'  # default

# 🧠 1. Mark Clinical by NETID
if 'netid' in hg_df.columns and 'netid' in slg_clinical_df.columns:
    clinical_netids = set(slg_clinical_df['netid'].dropna())
    hg_df.loc[hg_df['netid'].isin(clinical_netids), 'clinical_classification'] = 'Clinical'

# 🧠 2. Mark Non-Clinical by NETID
if 'netid' in slg_nonclinical_df.columns:
    nonclinical_netids = set(slg_nonclinical_df['netid'].dropna())
    hg_df.loc[hg_df['netid'].isin(nonclinical_netids), 'clinical_classification'] = 'Non-Clinical'

# 🧠 3. Mark Clinical by email match to Physician Survey
if 'campus_email_addr' in hg_df.columns and 'primary_email' in physician_df.columns:
    physician_emails = set(physician_df['primary_email'].astype(str).str.strip().str.lower())
    hg_df.loc[
        (hg_df['clinical_classification'] == 'Unclassified') &
        (hg_df['campus_email_addr'].isin(physician_emails)),
        'clinical_classification'
    ] = 'Clinical'

# ✅ Preview classification results
hg_df['clinical_classification'].value_counts()


,count
clinical_classification,
Unclassified,5028


Ran into issue here, none of the hg_df are matching. So this is prolly because of

1. netid column mismatch
Column names or values may differ across files. For example:

NetID vs netid vs net_id

One may contain uppercase, one lowercase

One file may be missing netid entirely

2.  Empty netid values
The slg_clinical_df or slg_nonclinical_df might not have usable NetIDs, or might have NaN.

3.  Whitespace or format issues
Leading/trailing spaces, differences in email domain formats, etc.

4.  Email columns not matching correctly
physician_df might use a different column than primary_email for actual work emails.

using this code to debug


In [ ]:
# Check if 'netid' exists and is clean
print("hg_df netid sample:", hg_df['netid'].dropna().unique()[:5])
print("slg_clinical_df netid sample:", slg_clinical_df.columns)
print("slg_nonclinical_df netid sample:", slg_nonclinical_df.columns)

# Show email column names in physician_df
print("physician_df columns:", physician_df.columns.tolist())

# Check overlap
if 'netid' in slg_clinical_df.columns:
    clinical_netids = set(slg_clinical_df['netid'].dropna().astype(str).str.strip().str.lower())
    overlap = hg_df[hg_df['netid'].isin(clinical_netids)]
    print(f"✅ Matches found in SLG Clinical: {len(overlap)}")

if 'netid' in slg_nonclinical_df.columns:
    nonclinical_netids = set(slg_nonclinical_df['netid'].dropna().astype(str).str.strip().str.lower())
    overlap = hg_df[hg_df['netid'].isin(nonclinical_netids)]
    print(f"✅ Matches found in SLG Non-Clinical: {len(overlap)}")


hg_df netid sample: ['aa1014' 'aa1330' 'aa1576' 'aa1917' 'aa2447']
slg_clinical_df netid sample: Index(['Email Address', 'Lname', 'Fname', 'UBHC_UCHC', 'School',
       'Department/School Division', 'Department/School Division_Revised_A',
       'Department/School Division_Revised_B', 'Institute', 'Track/Tenure',
       'Academic title/PGY', 'Residency Institution', 'Residency Program Name',
       'Residency Program Name_Revised', 'Clinical Staff Department/Division',
       'Population Category', 'FTE', 'Secondary Email', 'Middle Name',
       'UBHC_UCHC Management_Non-Management', 'UBHC_UCHC_DISCIPLINE', 'ID',
       'NPI_NUMBER', 'NetID', 'VERGE_UNIQUE_ID', 'SIY', 'Unnamed: 26'],
      dtype='object')
slg_nonclinical_df netid sample: Index(['Email Address', 'Lname', 'Fname', 'UBHC_UCHC', 'School',
       'Department/School Division', 'Department/School Division_Revised_A',
       'Department/School Division_Revised_B', 'Institute', 'Track/Tenure',
       'Academic title/PostDoc', '

In [ ]:
# 🔧 Rename NetID column in SLG Clinical/Non-Clinical to lowercase to match hg_df
slg_clinical_df.rename(columns={'NetID': 'netid'}, inplace=True)
slg_nonclinical_df.rename(columns={'NetID': 'netid'}, inplace=True)

# Also lowercase the values for matching
slg_clinical_df['netid'] = slg_clinical_df['netid'].astype(str).str.strip().str.lower()
slg_nonclinical_df['netid'] = slg_nonclinical_df['netid'].astype(str).str.strip().str.lower()


In [ ]:
# Reset the classification column to "Unclassified"
hg_df['clinical_classification'] = 'Unclassified'

# 1. Classify as Clinical
clinical_netids = set(slg_clinical_df['netid'].dropna())
hg_df.loc[hg_df['netid'].isin(clinical_netids), 'clinical_classification'] = 'Clinical'

# 2. Classify as Non-Clinical
nonclinical_netids = set(slg_nonclinical_df['netid'].dropna())
hg_df.loc[
    (hg_df['clinical_classification'] == 'Unclassified') &
    (hg_df['netid'].isin(nonclinical_netids)),
    'clinical_classification'
] = 'Non-Clinical'

# 3. Preview updated classification breakdown
hg_df['clinical_classification'].value_counts()


,count
clinical_classification,
Unclassified,2291
Non-Clinical,1734
Clinical,1003


After seperating and cleaning the data, we now understand how many are Unclassified, non-clincal, and clinical.

Clinical = 1003

Non- Clinical = 1734

Unclassified = 2291

We still need to categorize the unclassified. So now we can move onto another criteria using "email" from "Physician_df" and NPI ID.

Cleaning by "Email"


In [ ]:
# 📥 Step 7: Match remaining Unclassified people by email address

# First, make sure both emails are cleaned
hg_df['campus_email_addr'] = hg_df['campus_email_addr'].astype(str).str.strip().str.lower()
physician_df['PRIMARY_EMAIL'] = physician_df['PRIMARY_EMAIL'].astype(str).str.strip().str.lower()

# Get set of physician emails
physician_emails = set(physician_df['PRIMARY_EMAIL'].dropna())

# How many unclassified before email matching
print(f"🔎 Unclassified before email matching: {(hg_df['clinical_classification'] == 'Unclassified').sum()}")

# Match: If a person is Unclassified but their campus_email_addr is in physician emails → mark as Clinical
hg_df.loc[
    (hg_df['clinical_classification'] == 'Unclassified') &
    (hg_df['campus_email_addr'].isin(physician_emails)),
    'clinical_classification'
] = 'Clinical'

# ✅ After matching, show updated counts
print("\n✅ Updated classification counts:")
print(hg_df['clinical_classification'].value_counts())

# Also print how many remain Unclassified
print(f"\n🔎 Remaining Unclassified after email matching: {(hg_df['clinical_classification'] == 'Unclassified').sum()}")


🔎 Unclassified before email matching: 2291

✅ Updated classification counts:
clinical_classification
Unclassified    2214
Non-Clinical    1734
Clinical        1080
Name: count, dtype: int64

🔎 Remaining Unclassified after email matching: 2214


| Classification     | Before Email Match | After Email Match | Change |
|---------------------|--------------------|-------------------|--------|
| Clinical            | 1003               | 1080              | **+77** |
| Non-Clinical        | 1734               | 1734              | — |
| Unclassified        | 2291               | 2214              | **-77** |

Results:

Successfully classified 77 more people as Clinical just by matching their email to the Physician Survey.

The number of Unclassified entries dropped from 2291 ➔ 2214.

* Total Employees: 5028

* Clinical Classified: 1080

* Non-Clinical Classified: 1734

* Still Unclassified: 2214



Until now I have been doing simple cleaning by doing individual comparisons. Now rather than doing that, we should compare the employees with the similar employee (same person) but with slight differences.

* This will polish the dataset for the longer run rather than doing simple immediate cleaning.

When someone appears multiple times in hg_df (your RutgersHGEmployeeList), sometimes:

* Same netid, but different emails

* Same netid, but missing NPI in one row and present in another

* Typos, missing fields, or slight inconsistencies

We need to compare across multiple columns (not just netid) and merge the records intelligently.

If we find rows with the same netid:
Prefer the row with:

* Non-missing NPI_NUMBER

* A complete campus_email_addr

* More filled fields

Keep the most complete version

Drop the weaker/incomplete duplicates

New Plan:

1. Find duplicate NetIDs in hg_df

2. Compare fields: NPI, email, names, job title

3. Keep best rows (most non-null values)

4. Drop the others

In [ ]:
# 📦 Step: Smart Deduplication based on NetID

# 1. Count duplicates first
duplicate_netids = hg_df[hg_df.duplicated(subset=['netid'], keep=False)]
print(f"🔎 Total duplicate netid records found: {duplicate_netids.shape[0]}")

# 2. Define a helper function to score each row by how "complete" it is
def completeness_score(row):
    score = 0
    for col in ['npi_number', 'campus_email_addr', 'preferred_email_addr', 'first_name', 'last_name', 'employee_id']:
        if pd.notna(row.get(col)):
            score += 1
    return score

# 3. Apply the completeness score
hg_df['completeness_score'] = hg_df.apply(completeness_score, axis=1)

# 4. Sort by NetID and Completeness
hg_df = hg_df.sort_values(by=['netid', 'completeness_score'], ascending=[True, False])

# 5. Drop duplicate NetIDs, keeping the most complete version
hg_df = hg_df.drop_duplicates(subset=['netid'], keep='first')

# 6. Drop the helper column now
hg_df = hg_df.drop(columns=['completeness_score'])

# ✅ Final clean shape
print(f"✅ After smart deduplication, dataset shape: {hg_df.shape}")

# ✅ Preview
hg_df.head()


🔎 Total duplicate netid records found: 0
✅ After smart deduplication, dataset shape: (5028, 27)


,employee_id,netid,employee_name,last_name,first_name,middle_init,job_title,prime_ind,current_flag,fac_staff_ind,...,dept_level4,dept_level5,employment_status_code,employee_class,tenure_date,preferred_email_addr,campus_email_addr,npi_number,fte,clinical_classification
330,154389,aa1014,"Athanasakis,Andreas",Athanasakis,Andreas,NaN,CMC OPERATOR I,Y,Y,Staff,...,CINJ-GMP Facility,NaN,A,1,NaN,athanasakisandreas@yaho..com,aa1014@research.rutgers.edu,NaN,1.0,Non-Clinical
58,182194,aa1330,"Adedeji,Anuoluwapo J",Adedeji,Anuoluwapo,J,POST DOCTORAL ASSOC,Y,Y,Faculty,...,Pharm-Pharmacy Practice&Admin,NaN,A,1,NaN,anujoyadedeji@gmail.com,joy.adedeji@rutgers.edu,NaN,1.0,Unclassified
264,78203,aa1576,"Antony Jude,Anthonypillai Joseph",Antony Jude,Anthonypillai Joseph,NaN,ASST RESEARCH PROFESSOR CAL YR,Y,Y,Faculty,...,Pharm-Pharmacology&Toxicology,NaN,A,1,NaN,aantony@mail.med.upenn.edu,joseph.antonyjude@rutgers.edu,NaN,1.0,Non-Clinical
146,126370,aa1917,"Alghali-Ano,Adisa",Alghali-Ano,Adisa,NaN,PROG MGR,Y,Y,Staff,...,UBHC-Acute Psych Services,UBHC-Inpatient/CTR,A,1,NaN,adisaalghali@gmail.com,aa1917@rutgers.edu,NaN,1.0,Clinical
351,159930,aa2447,"Avula,Amulya",Avula,Amulya,NaN,HLTH PROGRAM ANALYST,Y,Y,Staff,...,NJMS-Medicine Admin,NJMS-Medicine-Infectious Disea,A,1,NaN,amulyaavula5@gmail.com,amulya.avula@rutgers.edu,NaN,1.0,Unclassified


This dropped all the irregular duplicates, by calcautaing a "completeness score"

+1 point if first_name exists

+1 point if last_name exists

+1 point if email exists

+1 point if npi_number exists

+1 point if employee_id exists

etc.

The row with the highest total points (i.e., most fields filled in) was kept.

** More filled fields = More reliable = Row is kept

** Fewer filled fields = Row is dropped


Reasoning:

The reason I did this was because if I blindy pick a duplicate, I might accidentally keep a row with missing information.

We want the most complete, most detailed record for each employee in the final file.

*  That's why we score rows by how much useful info they have, and keep the best one.

* Because you already deduplicated well earlier, you didn't even have duplicate NetIDs left.

So there was nothing extra to merge or score — that's why smart deduplication found 0 duplicates.
It just confirmed that your dataset is clean!

| Classification      | Count |
|---------------------|-------|
| Clinical             | 1080 |
| Non-Clinical         | 1734 |
| **Unclassified**     | **2214** |

Which is still wrong, we need to fix all NC and C.

These are employees who:

Were not found in SLG Clinical or Non-Clinical

Did not match any known physician survey emails

Might have missing or slightly inconsistent identifiers (names, emails, NPI)

(Not sure if this is normal or not)

.
.
.
.
.

Goal is to get 0 unclassified

Steps to get there:

1. Name + NPI Matching

Match by first name + last name + NPI number.

Some people may not match by email but they will match perfectly by name + NPI in Physician Survey.


2. More Aggressive Email Matching
Some employees might have a preferred email (preferred_email_addr) instead of campus email (campus_email_addr).

Match using both emails. (Personal email use?)

3. Manual Review of Remaining Edge Cases
After automatic matching, there will still be a few rows left.

You will manually:

Look at their job title

Look at their department

Look at NPI (if they have one)

Decide manually: Clinical or Non-Clinical.





For every Unclassified person in hg_df, if their:

first_name

last_name

npi_number

matches a row in the Physician Survey, we'll classify them as Clinical.

In [ ]:
# 📊 Step 8: Match by first_name + last_name + npi_number

# 1. Normalize columns in both dataframes
hg_df['first_name'] = hg_df['first_name'].astype(str).str.strip().str.lower()
hg_df['last_name'] = hg_df['last_name'].astype(str).str.strip().str.lower()
hg_df['npi_number'] = hg_df['npi_number'].astype(str).str.strip()

physician_df['FIRST_NAME'] = physician_df['FIRST_NAME'].astype(str).str.strip().str.lower()
physician_df['LAST_NAME'] = physician_df['LAST_NAME'].astype(str).str.strip().str.lower()
physician_df['NPI_NUMBER'] = physician_df['NPI_NUMBER'].astype(str).str.strip()

# 2. Build a set of (first, last, npi) tuples from physician_df
physician_tuples = set(
    physician_df[['FIRST_NAME', 'LAST_NAME', 'NPI_NUMBER']]
    .dropna()
    .itertuples(index=False, name=None)
)

# 3. Match hg_df rows that are unclassified and in the tuple set
mask_unclassified = hg_df['clinical_classification'] == 'Unclassified'

# Create tuple column in hg_df for comparison
hg_tuples = hg_df[['first_name', 'last_name', 'npi_number']].apply(tuple, axis=1)

# Update classification if match found
hg_df.loc[
    mask_unclassified & hg_tuples.isin(physician_tuples),
    'clinical_classification'
] = 'Clinical'

# ✅ Show updated classification counts
print("\n✅ Updated classification counts:")
print(hg_df['clinical_classification'].value_counts())

# 🔍 Show how many are still unclassified
print(f"\n🔍 Remaining Unclassified: {(hg_df['clinical_classification'] == 'Unclassified').sum()}")



✅ Updated classification counts:
clinical_classification
Unclassified    2214
Non-Clinical    1734
Clinical        1080
Name: count, dtype: int64

🔍 Remaining Unclassified: 2214


So i noticed nothing has changed. I've honestly no idea what im doing: sorry.

but this means that none of the Unclassified people matched the (first name, last name, NPI) tuples in the physician_df.

Heres what I'm thinking:

1. Most of the remaining Unclassfied people are non- clinical roles

2.  Their npi_number is missing or formatted inconsistently in hg_df

3. Their names might not match perfectly
If there are:

Typos

Middle initials used inconsistently

Diacritics or formatting issues
→ Matching on (first, last, NPI) won't work



Even after all this, I was thinking maybe I can compare with the other files that I have collected.


USed:

- SLG Clinical
_ SLG Non Clinical
- Phycisians Survery
- RutgersHGEmployeeList



Not Used:

- AllFacultyReport
- RBHS_Fac_Staff (Nick's File)
- PWAC Resident
- RU Health Postdocs

